In [1]:
#Q1 
#this year
# Constants and initial values
# Function to calculate the after-tax present value of the vaccine project
def calculate_npv(initial_investment, sales_per_unit, units_sold_annually, wage_rate_per_hour, 
                  hours_of_labor_annually, cost_per_ton_raw_material, tons_of_raw_material_annually, 
                  inflation_rate, corporate_tax_rate, depreciation, interest_rate, years):
    # Initialize list to hold the cash flows
    cash_flows = []

    # Calculate the cash flows for each year
    for year in range(1, years + 1):
        # Adjust for inflation
        sales = sales_per_unit * (1 + inflation_rate) ** (year - 1) * units_sold_annually
        labor_cost = wage_rate_per_hour * (1 + inflation_rate) ** (year - 1) * hours_of_labor_annually
        material_cost = cost_per_ton_raw_material * (1 + inflation_rate) ** (year - 1) * tons_of_raw_material_annually
        operating_income = sales - (labor_cost + material_cost)
        profit_before_tax = operating_income - depreciation
        tax = profit_before_tax * corporate_tax_rate
        profit_after_tax = profit_before_tax - tax
        cash_flow = profit_after_tax + depreciation # Add back depreciation as it's a non-cash charge
        cash_flows.append(cash_flow)
    
    # Calculate the present value of the cash flows
    npv = -initial_investment
    for i, cash_flow in enumerate(cash_flows, start=1):
        npv += cash_flow / (1 + interest_rate) ** i
    
    # Return the NPV rounded to the nearest dollar
    return round(npv)

# Define the parameters based on the provided scenario
initial_investment = 10000000
sales_per_unit = 3.30
units_sold_annually = 1000000
wage_rate_per_hour = 30
hours_of_labor_annually = 10000
cost_per_ton_raw_material = 100
tons_of_raw_material_annually = 100
inflation_rate = 0.10  # 10% inflation rate
corporate_tax_rate = 0.34
depreciation = initial_investment / 5
interest_rate = 0.03
years = 5

# Calculate the NPV
npv = calculate_npv(initial_investment, sales_per_unit, units_sold_annually, wage_rate_per_hour, 
                    hours_of_labor_annually, cost_per_ton_raw_material, tons_of_raw_material_annually, 
                    inflation_rate, corporate_tax_rate, depreciation, interest_rate, years)

print(f"The (after-tax) present value of the new vaccine project is: ${npv}")


The (after-tax) present value of the new vaccine project is: $4087435


In [5]:
#Q1 Matlab->Python
import numpy as np

# Costs
c = 30 * 10000 * (1 + 0.1) ** np.arange(5) + 100 * 100 * (1 + 0.1) ** np.arange(5)

# Revenue
r = 3.3 * 1000000 * (1 + 0.1) ** np.arange(5)

# Profit
p = r - c

# Tax calculation
tax = (p - 10e6 / 5 * np.ones(5)) * 0.34

# Profit after tax
p1 = p - tax

# Discounting future cash flows to present value
discounted_cash_flows = np.array([-10e6] + list(p1)) / (1.03) ** np.arange(6)

# Rounding the sum of discounted cash flows
result = round(np.sum(discounted_cash_flows))
print("Result:", result)


Result: 4087435


In [1]:
#Q2 主要会这个就行
def cash_flow_present_value(cash_flow, effective_rate):
    #"""Enter effective rate in percentage"""
    if isinstance(effective_rate, list) or isinstance(effective_rate, tuple):
        spr = [1] + [1 + r/100 for r in effective_rate]
        p = 0.
        for i, ai in enumerate(cash_flow):
            p += ai / (spr[i]) ** i
        return p
    r = 1 + effective_rate / 100
    res = 0
    for i, ai in enumerate(cash_flow):
        res += ai / r ** i
    return res

class Bond:
    #"""Implementation of a bond. A face value $30000, 9% bond that matures in 30 periods (years) that pays its coupons twice a period (year) should be entered as Bond(9, 30, 2, 300)"""
    def __init__(self, coupon_percentage, maturity_years, coupon_freq, face_value):
        self.percentage = coupon_percentage / 100
        self.maturity_years = maturity_years
        self.coupon_freq = coupon_freq
        self.face_value = face_value
        flow = [0] + [coupon_percentage / coupon_freq] * maturity_years * coupon_freq
        flow[-1] += face_value
        self.flow = tuple(flow)
    
    @property
    def coupon_payment(self):
        return self.percentage * self.face_value / self.coupon_freq

def QM_duration(cash_flow_or_bond, rates, compound_freq):
    #"""Enter a list of spot rates (in percentage) to calculate the quasi-modified duration.
    #Keep in mind if the price at rates = sk + y is P(y), then P'(0) = -Dq * P(0)

    #This trims the first term to keep consistent with the bond calculations. i.e. following the book. calculating the
    #quasimodified duration for (x0, x1, ..., xn) where x0 is the flow on year 0
    
    #You can actually feed a bond into this function and we will handle it for you"""
    
    if isinstance(cash_flow_or_bond, Bond):
        cash_flow = cash_flow_or_bond.flow
    else:
        cash_flow = cash_flow_or_bond
    
    assert len(cash_flow) <= len(rates) + 1, f"Wrong number of periods: we have {len(rates)} years of spot rates but {len(cash_flow)} years bond"

    # First calculate the pv of cash flow
    spr = [1] + [1 + r/100 for r in rates]
    res = 0
    for i, ai in enumerate(cash_flow):
        res += ai / (spr[i]) ** i
    PV = res

    x = cash_flow[1:]
    s = [1 + r/100 for r in rates]
    m = compound_freq
    n = len(cash_flow) - 1
    res = 0.
    for _k in range(n):
        k = _k + 1
        rs = (k/m) * x[_k] * (s[_k]) ** (-k-1)
        res += rs
    return res/PV

def immunize_parallel(bond1: Bond, bond2: Bond, obligation_cash_flow, rates, verbose: bool = False):
    # Calculate the PVob, Dob of obligation
    # Calculate the PV1, D1 of bond1
    # Calculate the PV2, D2 of bond2

    obligation = obligation_cash_flow

    # First do error checking
    num_years_total = max(bond1.maturity_years, bond2.maturity_years, len(obligation) - 1)
    enough_rates = len(rates) >= num_years_total
    if not enough_rates:
        raise ValueError(f"Expected at least {num_years_total} years of rates, found {len(rates)}")

    PVob = cash_flow_present_value(obligation, rates)
    PV1 = cash_flow_present_value(bond1.flow, rates)
    PV2 = cash_flow_present_value(bond2.flow, rates)
    
    Dob = QM_duration(obligation, rates, 1)
    D1 = QM_duration(bond1, rates, 1)
    D2 = QM_duration(bond2, rates, 1)

    if verbose:
        print(f"Duration of bond 1: {D1}")
        print(f"Duration of bond 2: {D2}")
        print(f"Duration of obligation: {Dob}")
        print(f"Present value of bond 1: {PV1}")
        print(f"Present value of bond 2: {PV2}")
        print(f"Present value of obligation: {PVob}")

    # Solve for
    # PV1   PV2     x1 = PVob
    # PV1D1 PV2D2   x2 = PVob Dob

    # Hence
    # x1 =  |PV1   PV2  |-1  |PVob    |
    # x2 =  |PV1D1 PV2D2|    |PVob Dob|
    det = PV1 * PV2 * D2 - PV2 * PV1 * D1
    if abs(det) < 1e-5:
        raise ValueError("This portfolio cannot be immunized - the equations have no solutions")

    x1 = 1/det * (PV2 * PVob * D2 - PVob * Dob * PV2)
    x2 = 1/det * (PV1 * PVob * Dob - PVob * D1 * PV1)

    if verbose:
        print(f"Buy {x1} units of bond 1 and {x2} units of bond 2")

    return x1, x2


## The inputs: in percentage
spot_rates = [2.25, 2.50, 2.75, 3.00, 3.25]
bond1 = Bond(6, 5, 1, 100)
bond2 = Bond(10, 4, 1, 100)
obligation = (0, 50000, 51000 + 51000, 52000 + 52000, 53000 + 53000, 54000)
immunize_parallel(bond1, bond2, obligation, spot_rates)


(-2033.5124975117003, 4842.659952631226)

In [9]:
#Q2 Matlab->Python
import numpy as np

# Defining the spot rates
s = np.array([225, 250, 275, 300, 325]) / 1e4

# Calculations for the first set of cash flows
pv1 = np.array([6] * 4 + [106]) / (1 + s) ** np.arange(1, 6)
P1 = np.sum(pv1)
dP1dlambda = np.array([6] * 4 + [106]) * np.arange(1, 6) * (1 + s) ** -np.arange(2, 7)
D1 = np.sum(dP1dlambda) / P1

# Calculations for the second set of cash flows
pv2 = np.array([10] * 3 + [110]) / (1 + s[:4]) ** np.arange(1, 5)
P2 = np.sum(pv2)
dP2dlambda = np.array([10] * 3 + [110]) * np.arange(1, 5) * (1 + s[:4]) ** -np.arange(2, 6)
D2 = np.sum(dP2dlambda) / P2

# Calculations for the observed cash flows
ob = np.array([50000, 102000, 104000, 106000, 54000])
P_ob = np.sum(ob / (1 + s) ** np.arange(1, 6))
D_ob = np.sum(ob * np.arange(1, 6) * (1 + s) ** -np.arange(2, 7)) / P_ob

# Solving the linear equations
x = np.round(np.linalg.solve([[P1, P2], [P1 * D1, P2 * D2]], [P_ob, P_ob * D_ob]), 3)

print("x:", x)


x: [-2033.512  4842.66 ]


In [ ]:
#Q3
#2*79.66-100=59.32

In [10]:
#Q3 matlab->Python
import numpy as np

# Given values
P1 = 79.66
P2 = 100
C1 = 0.03
C2 = 0.06
F = 100
T = 10

# Solving the system of linear equations
X = np.linalg.solve([[F, F], [C1, C2]], [F, 0])

# Calculating the price P
P = np.dot(X, [P1, P2])

# Calculating the yield to maturity S
S = (F / P) ** (1 / T) - 1

print("X:", X)
print("P:", P)
print("S:", S)


X: [ 2. -1.]
P: 59.31999999999999
S: 0.05361000442717456


In [2]:
#Q4
def new_paymt(P, n1, m, r1, r2, n2): 
    A1 = r1/m*P*(1-1/(1+r1/m)**(n1*m))**(-1)
    print('Orginal Payment:', A1)
    P1 = A1/(r1/m)*(1-1/(1+r1/m)**((n1-n2)*m))
    A2 = r2/m*P1*(1-1/(1+r2/m)**((n1-n2)*m))**(-1)
    return A2

new_paymt(P=1000000, n1=30, m=12, r1=0.02, r2=0.05, n2=5)

Orginal Payment: 3696.194726888144


5097.880172055114

In [11]:
#Q4 matlab->Python
# Initial loan parameters
P1 = 1000000
n1 = 30 * 12  # 30 years in months
r1 = 0.02 / 12  # Monthly interest rate

# Monthly payment calculation for the first loan
A = (r1 * (1 + r1) ** n1 * P1) / ((1 + r1) ** n1 - 1)

# New loan parameters
n2 = 25 * 12  # 25 years in months
r2 = 0.05 / 12  # New monthly interest rate

# Recalculating the principal for the new loan
P = (A / r1) * (1 - 1 / (1 + r1) ** n2)

# Monthly payment calculation for the new loan
A1 = (r2 * (1 + r2) ** n2 * P) / ((1 + r2) ** n2 - 1)

print("Monthly payment for the first loan (A):", A)
print("Recalculated principal for the new loan (P):", P)
print("Monthly payment for the new loan (A1):", A1)


Monthly payment for the first loan (A): 3696.194726888144
Recalculated principal for the new loan (P): 872043.6220406473
Monthly payment for the new loan (A1): 5097.880172055114


In [3]:
#Q5
import math
#P = mortegage value n1 = original terms, n2= rest terms, r1 = ir1, r2 =ir2
def ext_term(P, n1, m, r1, r2, n2): 
    A1 = r1/m*P*(1-1/(1+r1/m)**(n1*m))**(-1)
    P1 = A1/(r1/m)*(1-1/(1+r1/m)**((n1-n2)*m))
    a = P1 * r2/m / A1
    b = 1 - a
    t = -math.log(b, (1+r2/m))
    return t - (n1-n2)*m

ext_term(P=1000000, n1=30, m=12, r1=0.02, r2=0.05, n2=5)

680.5225038849858

In [14]:
#Q6
# Parameters as per the user's description
initial_cost = 16000  
replacement_cost = 16000
annual_increase = 1000
interest_rate = 0.06
operating_cost_first_year = 2000

# Define the operating costs for a maximum of 30 years
years = 30
operating_costs = [operating_cost_first_year + i * annual_increase for i in range(years)]

# Function to calculate present value of costs
def present_value(cost, rate, year):
    return cost / ((1 + rate) ** year)

# Calculate the present value of the operating costs for each year
present_values_operating = [present_value(oc, interest_rate, year) for year, oc in enumerate(operating_costs, start=1)]

# Function to calculate the total cost for a given replacement period n
def total_cost(n):
    # Present value of initial cost
    pv_initial_cost = present_value(initial_cost, interest_rate, 0)
    # Sum of present values of operating costs up to year n
    pv_operating_costs = sum(present_values_operating[:n])
    # Total present value cost of owning the machine for n years (including replacement every n years indefinitely)
    # is the initial cost plus operating costs times the series sum of (1 - (1 + r)^(-n))^-1
    # This is the present value annuity factor for a perpetuity with payments every n years
    annuity_factor = (1 - (1 + interest_rate)**(-n))**(-1)
    total_pv_cost = (pv_initial_cost + pv_operating_costs) * annuity_factor
    return total_pv_cost

# Initialize the minimum cost to a large number and optimal n to None
min_cost = float('inf')
optimal_n = None

# Check for the optimal replacement period from 1 to 30 years
for n in range(1, years + 1):
    cost_n = total_cost(n)
    if cost_n < min_cost:
        min_cost = cost_n
        optimal_n = n

optimal_n, min_cost


(6, 126403.43124625734)

In [14]:
#Q6 
import numpy as np

# Parameters
interest_rate = 0.06

# Initialize the array A
A = np.zeros(49)

# Loop to calculate NPV for each case
for i in range(2, 51):
    a = [2000] + [2000 + j * 1000 for j in range(1, i)] + [2000 + i * 1000 + 16000]
    b = [16000] + a * 50
    npv = np.sum(b / np.power(1 + interest_rate, np.arange(len(b))))
    A[i - 2] = npv

# Finding the minimum NPV and corresponding index
min_value = np.min(A)
min_index = np.argmin(A) + 2  # adding 2 to match the MATLAB indexing

# Adjusting for MATLAB's 1-based indexing
adjusted_min_index = min_index + 1

# Output the result
adjusted_min_index



6

In [ ]:
#Q7 sentence

In [4]:
#Q8  
def allocate_a(n, rb, rm, rf):   # $n dollars, a in market portfolio, n-a in risk-free asset, return a
    # n*rb = a*rm + (n-a)*rf
    return n*(rb-rf) / (rm-rf)

allocate_a(n=1000, rb=0.18, rm=0.12, rf=0.02)
     

1600.0000000000002

In [16]:
#Q9-11
import numpy as np
from numpy import ndarray
class OptimizeStockPortfolio:
    def __init__(self, expected_returns: list[float], covariances: np.ndarray[np.float64]):
        #"""Solves the Markovitz problem. Refer to the example below"""
        r, w = covariances.shape
        self.n = len(expected_returns)

        if r != w or r != self.n:
            raise ValueError(f"Expected number of stocks equal number of correlations... got number of stocks: {self.n}, correlation array: {r}x{w}")

        if not np.all(covariances - covariances.T < 1e-6): #type: ignore
            raise ValueError(f"Expected the covariance to be a symmetric matrix")

        # if np.all(np.abs(covariances.diagonal() - 1) < 1e-6): #type: ignore
        #     print("All diagonal entries are 1, did you enter the correlation matrix?")

        self.expecteds = np.array(expected_returns)
        self.covariances = np.array(covariances)

    def returns(self, weights: ndarray[np.float64]) -> tuple[float,float]: #type: ignore
        r = weights.dot(self.expecteds)
        sd = weights.T @ self.covariances @ weights
        return r, sd #type: ignore

    def solve_analytical(self):
        n, = self.expecteds.shape
        A = np.array(self.covariances)
        b = np.ones((n,))

        A_inv: ndarray[np.float64]

        try:
            A_inv = np.linalg.inv(A)
        except LinAlgError:
            raise ValueError("The matrix is not invertible. Try solving it the numerical way")

        x = A_inv @ b
        best_weight = x[:n] / np.sum(x[:n])
        best_r, best_sd = self.returns(best_weight)

        print(f"Best average return: {best_r}")
        print(f"Minimum standard deviation: {best_sd}")
        print(f"Best weights: {best_weight}")

        return best_weight

    def solve_analytical_target_return(self, target_mean_return: float):
       # """Solves the Markowitz problem analytically. Enter the target mean return r-bar in percentage"""
        r_bar = target_mean_return
        n, = self.expecteds.shape

        # There are n + 2 variables in total, the equations (in compact form) are:
        # w1 + ... + wn = 1
        # w1r1 + w2r2 + ... + wnrn = r_bar
        # C w - l r - m = 0
        # where the n + 2 variables are w1 ... wn and l and m

        # So in the spirit of solving system of linear equations, we make the matrix A and vector b so that we solve Ax = b

        A = np.zeros((n+2, n+2))
        b = np.zeros((n+2,))

        # First equation
        A[0, :n] = 1
        b[0] = 1

        # Second equation
        A[1, :n] = self.expecteds
        b[1] = r_bar

        # Covariance equations
        A[2:, :n] = self.covariances
        A[2:, n] = -self.expecteds
        A[2:, n+1] = -1

        # Solve equation the lazy way:
        A_inv: np.ndarray[np.float64]

        try:
            A_inv = np.linalg.inv(A)
        except LinAlgError:
            raise ValueError("The matrix is not invertible. Try solving it the numerical way")

        x = A_inv @ b
        best_weight = x[:n]
        best_r, best_sd = self.returns(best_weight)

        print(f"Best average return: {best_r}")
        print(f"Minimum standard deviation: {best_sd}")
        print(f"Best weights: {best_weight}")

        return best_weight

    def solve_tan_analytical(self, risk_free_rate: float):
        n, = self.expecteds.shape
        A = np.array(self.covariances)
        b = self.expecteds - risk_free_rate

        A_inv: ndarray[np.float64]

        try:
            A_inv = np.linalg.inv(A)
        except LinAlgError:
            raise ValueError("The matrix is not invertible. Try solving it the numerical way")

        x = A_inv @ b
        best_weight = x[:n] / np.sum(x[:n])
        best_r, best_sd = self.returns(best_weight)

        print(f"Best average return: {best_r}")
        print(f"Minimum standard deviation: {best_sd}")
        print(f"Best weights: {best_weight}")

        return best_weight

def markowitz_analytical(expected_returns, covariances):
    #"""Solves the classical markowitz problem by using approximation approach"""
    s = OptimizeStockPortfolio(expected_returns, covariances)
    return s.solve_analytical()

def markowitz_riskfree_analytical(expected_returns, covariances, risk_free):
    #"""Solves markowitz problem with risk free rate. Enter risk free rate in percentage"""
    s = OptimizeStockPortfolio(expected_returns, covariances)
    return s.solve_tan_analytical(risk_free)

def markowitz_fixed_mean_analytical(expected_returns, covariances, target_mean_return):
    #"""Solves markowitz problem with a target mean return. Enter risk free rate in percentage"""
    s = OptimizeStockPortfolio(expected_returns, covariances)
    return s.solve_analytical_target_return(target_mean_return)

In [15]:
# Q9
returns = np.array([69, 65, 44, 36, 41])

covs = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])

print("Q9 - aim for 80% return")
w = markowitz_fixed_mean_analytical(returns, covs, 80)
print(w[0] * 10000)

print("\n\nQ10")
markowitz_analytical(returns, covs)

print("\n\nQ11")
w = markowitz_riskfree_analytical(returns, covs, 4)
print(w[3])


Q9 - aim for 80% return
Best average return: 79.99999999999996
Minimum standard deviation: 0.1618979763761178
Best weights: [ 0.74492302  0.89403309 -0.5980176   0.30411715 -0.34505565]
7449.230162571025


Q10
Best average return: 53.00171526586621
Minimum standard deviation: 0.12381360777587187
Best weights: [ 0.0971984   0.53230417 -0.15208691  0.60777587 -0.08519154]


Q11
Best average return: 101.36131335760292
Minimum standard deviation: 0.2460047653270432
Best weights: [ 1.25740922  1.18023663 -0.95084243  0.06385933 -0.55066275]
0.0638593297862414


In [17]:
#Q9 matlab ->Python
import numpy as np

# Covariance matrix V
V = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])

# Expected returns of assets
r_bar = np.array([0.69, 0.65, 0.44, 0.36, 0.41])

# Constructing the augmented matrix for the linear system
A = np.vstack([
    np.hstack([V, -r_bar[:, np.newaxis], -np.ones((5, 1))]),
    np.hstack([r_bar, [0], [0]]),
    np.hstack([np.ones(5), [0], [0]])
])

# Constructing the right-hand side of the equation
b = np.append(np.zeros(5), [0.8, 1])

# Solving the linear system
solution = np.linalg.solve(A, b)

# Extracting the weights and normalizing them to sum to 1
weights = solution[:5] / sum(solution[:5])

weights


array([ 0.74492302,  0.89403309, -0.5980176 ,  0.30411715, -0.34505565])

In [2]:
#Q10 matlab ->Python
import numpy as np

# Covariance matrix V
V = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])

# Augmenting the covariance matrix with ones to account for the constraint
augmented_matrix = np.vstack([V, np.ones(V.shape[1])])
augmented_matrix = np.hstack([augmented_matrix, np.ones((augmented_matrix.shape[0], 1))])
augmented_matrix[-1, -1] = 0

# Right-hand side vector
b = np.zeros(V.shape[1] + 1)
b[-1] = 1

# Solving the linear system
solution = np.linalg.solve(augmented_matrix, b)

# Extracting the weights
weights = solution[:-1]

weights, np.dot(weights.T, np.dot(V, weights))


(array([ 0.0971984 ,  0.53230417, -0.15208691,  0.60777587, -0.08519154]),
 0.12381360777587194)

In [17]:
#10  gpt生成
import numpy as np
from scipy.optimize import minimize

# Define the variance-covariance matrix
var_covar_matrix = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])

# Function to calculate portfolio variance
def calculate_portfolio_variance(weights):
    return weights.T @ var_covar_matrix @ weights

# Constraint for the weights to sum up to 1
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# Initial guess for the weights
initial_weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

# Bounds for the weights, allowing short selling
bounds = [(-1, 1) for _ in range(len(initial_weights))]

# Run the optimizer to minimize the portfolio variance
optimal_portfolio = minimize(calculate_portfolio_variance, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)

# The optimal weights
optimal_weights = optimal_portfolio.x

# The minimum portfolio variance
min_portfolio_variance = optimal_portfolio.fun

# Output the results
optimal_weights, min_portfolio_variance


(array([ 0.09784196,  0.53081372, -0.15176843,  0.60785143, -0.08473869]),
 0.12381379769561046)

In [15]:
#Q11 matlab->Python
import numpy as np

# Define the covariance matrix V and expected returns r_bar
V = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])
r_bar = np.array([0.69, 0.65, 0.44, 0.36, 0.41])
r_f = 0.04

# Solve for weights of one fund
v_1 = np.linalg.solve(V, r_bar - r_f)
w_1 = v_1 / sum(v_1)

# Mean return
MR1 = r_bar.T @ w_1
Var1 = w_1.T @ V @ w_1

# Desired mean
Desired_mean = 0.8

# Find weights of portfolio with desired mean
alpha = (Desired_mean - MR1) / (r_f - MR1)
w_2 = (1 - alpha) * w_1

# Variance of portfolio (for one fund s.d. = (1 - alpha) s.d. of one fund)
Var = (1 - alpha)**2 * w_1.T @ V @ w_1
Var2 = w_2.T @ V @ w_2

w_1, MR1, Var1, w_2, MR1, Var, Var2


(array([ 1.25740922,  1.18023663, -0.95084243,  0.06385933, -0.55066275]),
 1.0136131335760286,
 0.24600476532704305,
 array([ 0.98153052,  0.92128979, -0.74222525,  0.04984843, -0.42984598]),
 1.0136131335760286,
 0.1498986967558755,
 0.14989869675587547)

In [10]:
#Q12 Matlab->Python
import numpy as np

# Define the variables
var_v = 0.04
beta = np.array([1.16, 0.70, 1.15, 0.98, 1.01]).reshape(-1, 1)  # Reshape to column vector
cov_v = beta * var_v

# Define the matrix V
V = np.array([[0.21, 0.12, 0.17, 0.13, 0.16],
              [0.12, 0.15, 0.12, 0.10, 0.12],
              [0.17, 0.12, 0.19, 0.14, 0.15],
              [0.13, 0.10, 0.14, 0.15, 0.14],
              [0.16, 0.12, 0.15, 0.14, 0.21]])

# Construct the matrix and vector for the linear system
A = np.block([[2 * V, -np.ones((5, 1))],
              [np.ones((1, 5)), np.zeros((1, 1))]])
b = np.vstack([2 * cov_v, [[1]]])

# Solve the linear system
OPW_12 = np.linalg.solve(A, b)

# Print the result
print(OPW_12)


[[ 0.16576329]
 [ 0.34497427]
 [-0.00713551]
 [ 0.56686106]
 [-0.07046312]
 [ 0.18202401]]


In [11]:
#Q13 matlab->Python
# Updated data
V = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])
r_bar = np.array([0.69, 0.65, 0.44, 0.36, 0.41]).reshape(-1, 1)
var_v = 0.04
beta = np.array([1.16, 0.70, 1.15, 0.98, 1.01]).reshape(-1, 1)
cov_v = beta * var_v

# Constructing the matrices for the linear system
A = np.block([
    [2 * V, -np.ones((5, 1)), -r_bar],
    [np.ones((1, 5)), 0, 0],
    [r_bar.T, 0, 0]
])
b = np.vstack((2 * cov_v, 1, 0.1))

# Solving the linear system
OPW_13 = np.linalg.solve(A, b)
OPW_13


array([[-0.81944149],
       [-0.20522414],
       [ 0.67113585],
       [ 1.02873331],
       [ 0.32479647],
       [ 0.40946389],
       [-0.42911796]])

In [9]:
#Q14 
import numpy as np
from scipy.optimize import fsolve

# Function to calculate the price of a bond given its yield to maturity (YTM)
def bond_price(face_value, coupon_rate, n_periods, YTM):
    coupon = face_value * coupon_rate / 2  # Semi-annual coupon payment
    discount_factors = [(1 + YTM / 2)**-i for i in range(1, n_periods + 1)]
    price = sum(coupon * df for df in discount_factors) + face_value * discount_factors[-1]
    return price

# Function to calculate the yield to maturity of a bond
def bond_yield_to_maturity(price, face_value, coupon_rate, n_periods):
    # Function to be solved for the yield (the difference between market price and calculated price)
    func = lambda YTM: bond_price(face_value, coupon_rate, n_periods, YTM) - price
    # Initial guess for the yield (close to the coupon rate)
    YTM_initial_guess = coupon_rate
    # Solve for the yield to maturity
    YTM, = fsolve(func, YTM_initial_guess)
    return YTM

# Given values for the 5-year bond
price_5_year = 102  # Given market price of the 5-year bond
face_value = 100
coupon_rate_5_year = 0.04
n_periods_5_year = 5*2   # 5 years, semi-annual coupons

# Calculate the YTM for the 5-year bond
YTM_5_year = bond_yield_to_maturity(price_5_year, face_value, coupon_rate_5_year, n_periods_5_year)

# Now, calculate the price of the 10-year bond using the YTM of the 5-year bond
coupon_rate_10_year = 0.10
n_periods_10_year = 10*2  # 10 years, semi-annual coupons

# Calculate the price of the 10-year bond using the YTM of the 5-year bond
price_10_year = bond_price(face_value, coupon_rate_10_year, n_periods_10_year, YTM_5_year)

# Round the price to two decimal places
price_10_year_rounded = round(price_10_year, 2)

print(YTM_5_year)
price_10_year_rounded


0.035598064088857954


153.79

In [1]:
#Q14 (use this)
import numpy as np

def IRRPoly(cash_flow, m):
    coefficients = np.flip(cash_flow)
    roots = np.roots(coefficients)
    irr_poly = np.real(roots[(roots >= 0)  & (np.iscomplex(roots) == False)])[0]
    irr = m*(1 - irr_poly) / irr_poly
    return irr

# Bond characteristics
face_value = 100
coupon_payment = 4  #yearly
payments_per_year = 2
years_to_maturity = 5
current_price = 102

# Cash flows for the bond

cash_flows = (coupon_payment / payments_per_year)*np.ones(years_to_maturity * payments_per_year)
cash_flows[-1] += face_value
print(cash_flows)
cash_flows = np.insert(cash_flows, 0, -current_price)
print(cash_flows)

# Find YTM using IRRPoly function
ytm = IRRPoly( cash_flows, payments_per_year)

# Convert to a percentage and display the result
ytm_percentage = ytm * 100
print(f"The bond's yield to maturity is {ytm_percentage:.2f}%")

def bond_price(F, C, lamb, m, n):
    '''
    F is the face value of the bond
    C is the yearly coupon payment
    lamb is the yield of the bond 
    m is the number of coupon payments per year
    n is the number of coupon payments remaning
    '''
    r = lamb / m  # Semi-annual yield
    discount_factor = (1 + r) ** n
    bond_price = (C / r) * (1 - 1 / discount_factor) + F / discount_factor
    return bond_price

face_value = 100
coupon_rate = 0.1
periods=10
frequency = 2
coupon_payment_per_period = face_value * coupon_rate / frequency
bond_price(face_value, coupon_payment_per_period, ytm, frequency, periods * frequency)

[  2.   2.   2.   2.   2.   2.   2.   2.   2. 102.]
[-102.    2.    2.    2.    2.    2.    2.    2.    2.    2.  102.]
The bond's yield to maturity is 3.56%


153.78888729414587

In [18]:
#Q15
# Invest to pay off mortgage FPQ15
# Terms!!! Make sure it in month or year!!!
def inv_payoff(int_rate,terms,growth_rate,priciple):
    M = priciple*(int_rate/(1-(1+int_rate)**(-terms)))
    PFV = M*(((1+growth_rate)**terms)-1)/growth_rate
    FV_I = priciple*(1+growth_rate)**terms
    FV = FV_I - PFV
    print(FV)
    return FV
inv_payoff(0.0025,360,0.01,500000)

10607365.783267662


10607365.783267662

In [12]:
#Q15 matlab->Python
n = 30 * 12  # total number of months
r = 0.03 / 12  # monthly interest rate
P = 500000  # principal amount
A = ((r * (1 + r) ** n) * P) / ((1 + r) ** n - 1)  # monthly payment calculation

for i in range(n):
    P = P * 1.01 - A  # updating the principal with interest and subtracting the monthly payment

P  # remaining balance after last payment


10607365.78326779

In [21]:
#Q16
def sum_sharpe_ratios(num_assets):
    return sum(range(1, num_assets + 1))

def optimal_portfolio_weight(num_assets):
    return 1 / sum_sharpe_ratios(num_assets)

num_assets = 100

optimal_weight = optimal_portfolio_weight(num_assets)
optimal_weight_percentage = optimal_weight * 100
optimal_max_sharpe = optimal_weight_percentage * 100

print("The optimal portfolio weight for the risky asset with the maximum Sharpe ratio is approximately:",optimal_max_sharpe) 

The optimal portfolio weight for the risky asset with the maximum Sharpe ratio is approximately: 1.9801980198019802


In [13]:
#Q16 matlab->Python 最后要*100
import numpy as np
import sympy as sp

# Define the symbolic variables
var, rf = sp.symbols('var rf')

# Create the risk values rk
rk = np.linspace(0.01, 1, 100) * var + rf

# Create the diagonal matrix with var^2 and solve the linear equation
a = sp.Matrix(np.diag(np.ones(100) * var**2)).LUsolve(sp.Matrix(rk - rf))

# Calculate the ratio of the last element to the sum of all elements
ratio = a[-1] / sum(a)
ratio


0.0198019801980198

In [22]:
#Q17
# constant-proportion rebalancing FPQ17

def Cons_propo_rebal(prop_of_cash,prop_of_stock,days,initial):
    stock = prop_of_stock * initial
    cash = prop_of_cash * initial
    for i in range(10):
        Ex = ((stock)/2 + 2*(stock))/2
        stock = prop_of_stock*(Ex+cash)
        cash = prop_of_cash*(Ex+cash)
    E = stock + cash
    print(E)
    return E

Cons_propo_rebal(0.5,0.5,10,100)

324.7321025468409


324.7321025468409

In [16]:
#Q17 matlab->Python 还要检查
import numpy as np
import random

# Initialize an array of zeros
a = np.zeros(1000000)

# Main loop for simulation
for j in range(1000000):
    p = 100
    for i in range(10):
        if random.randint(1, 2) == 2:
            p = (p / 2) * 2 + (p / 2)
        else:
            p = (p / 2) / 2 + (p / 2)
    a[j] = p

# Calculate the mean
mean_value = np.mean(a)
print("Mean value:", mean_value)

# Additional calculation
p = 100
for i in range(10):
    p = (1 / 2) * (3 * p / 2) + (1 / 2) * (3 * p / 4)
print("Final value of p:", p)


Mean value: 324.8531931455612
Final value of p: 324.7321025468409


In [23]:
#Q18
from scipy.optimize import minimize

bonds = [
    Bond(0, 5, 1, 100),
    Bond(0, 10, 1, 100),
    Bond(0, 15, 1, 100),
    Bond(0, 20, 1, 100),
    Bond(0, 25, 1, 100),
    Bond(0, 30, 1, 100)
]

yield_curves = [1, 2.5, 2.5, 3, 3, 3.1]

def bond_price(bond, yield_to_maturity: float, remaining_periods = None):
    #"""Calculate the price of the bond based on its yield to maturity. The price and yield operations should be inverses to each other, given the same number of remaining periods until maturity.
    #Please enter your yield to maturity in percentages"""
    if remaining_periods is None:
        remaining_periods = bond.coupon_freq * bond.maturity_years
    
    m = bond.coupon_freq
    F = bond.face_value
    C = bond.percentage * bond.face_value
    y = yield_to_maturity / 100
    n = remaining_periods

    fac = 1 / (1 + y / m) ** n
    return F * fac + C / y * (1 - fac)

def treasury_yield_my_ass(bonds, yield_curves):
    prices = [bond_price(bond, rate) for bond, rate in zip(bonds, yield_curves)]

    # Lets say we have these weights
    s = [1, 0, 0, 0, 0, 0]

    def optimize_this(s):
        cost = sum(a * b for a, b in zip(s, prices))

        # In five years let's say we sell eveything
        y5 = 100 * s[0]
        for i in range(5):
            y5 += s[i+1] * prices[i]
        # We want biggest profit/cost ratio, hence smallest cost/profit
        return cost/y5

    ms = minimize(optimize_this, [1, 0, 0, 0, 0, 0], bounds = [(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)])

    weights = ms.x
    return 1/optimize_this(weights)

five_year_max_return_ratio = treasury_yield_my_ass(bonds, yield_curves)

# Do the five year thing twice
1e5 * five_year_max_return_ratio * five_year_max_return_ratio

155515.26561194478

In [4]:
#Q18 matlab->Python
import numpy as np

C = 0
F = 1
lambda_ = np.array([3.1, 3, 3, 2.5, 2.5, 1]) * 0.01
m = 1
n = np.array([30, 25, 20, 15, 10, 5])

P = F / ((1 + lambda_ / m) ** n)

# Iterating through the first five elements to calculate the ratio
for i in range(5):
    print(P[i + 1] / P[i])

# Calculating the specific value as per the last line of your code
result = 100000 * P[3] / P[2] * P[3] / P[2]
print("Result:", result)


1.1935190104692306
1.1592740743
1.2470575993591664
1.1314082128906244
1.217956521038558
Result: 155515.26561194472


In [ ]:
#7,12(存疑),13(存疑),14(存疑) 全对